# Imports

In [1]:
import os
from skimage import io
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split

### Hyperparams

In [2]:
num_px = 64
learning_rate = 0.005
num_iterations = 2000
print_cost = True
limit = 400

### Load images

In [3]:
def load_images(path, num_px=64, limit=100000000):
    files = os.listdir(path)

    images = []
    index = 0
    for file in files:
        if index > limit:
            break
        image = io.imread(os.path.join(path, file))
        images.append(resize(image, (num_px, num_px), mode='constant'))
        index += 1

    return np.array(images)


In [4]:
target_images = load_images('../data/images/target', num_px, limit=limit)

In [5]:
other_images = load_images('../data/images/others', num_px, limit=limit)

In [6]:
# np.save('../data/target_images_vec.npy', target_images)
# np.save('../data/other_images_vec.npy', other_images)

### Create dataset

In [7]:
target_images_flatten = target_images.reshape(target_images.shape[0], -1)
other_images_flatten = other_images.reshape(other_images.shape[0], -1)

target_values = np.ones((len(target_images), 1))
other_values = np.zeros((len(other_images), 1))

X = np.concatenate((target_images_flatten, other_images_flatten), axis=0)
y = np.concatenate((target_values, other_values), axis=0)

#### Split dataset into train set and test set

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train = X_train.T
y_train = y_train.T
X_test = X_test.T
y_test = y_test.T

### Helper functions

### Sigmoid function

In [9]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = (1 / ( 1 + np.exp(-z)))
    return s

In [10]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    w = np.zeros((dim, 1))
    b = 0
    
    return w, b

In [11]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    """
    
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T,X)+b)                # compute activation
    cost = (-1/m) * np.sum((Y * np.log(A)) + ((1 - Y) * np.log(1 - A)))     # compute cost
    
    #compute gradients
    dw = np.dot(X,(A-Y).T) / m
    db = np.sum(A-Y) / m

    cost = np.squeeze(cost)
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [12]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    """
    
    costs = []
    for i in range(num_iterations):
        
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        #update weights and b
        w = w - learning_rate * dw
        b = b - learning_rate * db
    
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [13]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[0,i] > 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

### Actual predicition

In [14]:
print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(y_test.shape))

w, b = initialize_with_zeros(len(X_train))
parameters, grads, costs = optimize(w, b, X_train, y_train, num_iterations, learning_rate, print_cost)

w = parameters["w"]
b = parameters["b"]

y_prediction_test = predict(w, b, X_test)
y_prediction_train = predict(w, b, X_train)


print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
print("train accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100))



train_set_x shape: (12288, 641)
train_set_y shape: (1, 641)
test_set_x shape: (12288, 161)
test_set_y shape: (1, 161)
Cost after iteration 0: 0.693147
Cost after iteration 100: 0.225528
Cost after iteration 200: 0.198166
Cost after iteration 300: 0.181116
Cost after iteration 400: 0.168456
Cost after iteration 500: 0.158212
Cost after iteration 600: 0.149525
Cost after iteration 700: 0.141950
Cost after iteration 800: 0.135219
Cost after iteration 900: 0.129159
Cost after iteration 1000: 0.123649
Cost after iteration 1100: 0.118600
Cost after iteration 1200: 0.113945
Cost after iteration 1300: 0.109631
Cost after iteration 1400: 0.105617
Cost after iteration 1500: 0.101870
Cost after iteration 1600: 0.098361
Cost after iteration 1700: 0.095067
Cost after iteration 1800: 0.091968
Cost after iteration 1900: 0.089048
test accuracy: 91.92546583850931 %
train accuracy: 98.12792511700468 %


In [15]:
X_train

array([[ 1.        ,  1.        ,  0.99607843, ...,  0.37916667,
         1.        ,  0.16369485],
       [ 1.        ,  1.        ,  1.        , ...,  0.31878064,
         1.        ,  0.16761642],
       [ 1.        ,  1.        ,  1.        , ...,  0.22858456,
         1.        ,  0.14800858],
       ..., 
       [ 1.        ,  1.        ,  1.        , ...,  0.08235294,
         1.        ,  0.60101103],
       [ 1.        ,  1.        ,  1.        , ...,  0.09803922,
         1.        ,  0.56963848],
       [ 1.        ,  1.        ,  1.        , ...,  0.10980392,
         1.        ,  0.48728554]])